<center>
    <h1> Assignment 6 </h1>
</center>

In [25]:
from DATA225utils import make_connection, dataframe_query
import pandas as pd
from DATA225utils import make_connection, dataframe_query
import csv

In [26]:
# try to connect to the database, other wise create the database and connect
try:
    conn = make_connection(config_file = './football.ini')
except:
    from create_schema import makedb
    makedb()
    conn = make_connection(config_file = './football.ini')

cursor = conn.cursor()

In [27]:
conn = make_connection(config_file = './football.ini')
cursor = conn.cursor()

In [28]:
df_appearances = pd.read_csv('appearances.csv')
df_appearances['ID'] = range(1, len(df_appearances) + 1)

In [29]:
def getPositionName(abbr):
    """
        Converts position acronyms to their full names
    """
    positions = {
        'GK':  'Goalkeeper',
        'DR':  'Right Defender',
        'DC':  'Central Defender',
        'DL':  'Left Defender',
        'DMC': 'Defensive Midfielder, Center',
        'AMR': 'Attacking Midfielder, Right',
        'AMC': 'Attacking Midfielder, Center',
        'AML': 'Attacking Midfielder, Left',
        'FW':  'Forward',
        'Sub': 'Substitute',
        'MR':  'Midfielder, Right',
        'MC':  'Midfielder, Center',
        'ML':  'Midfielder, Left',
        'FWR': 'Forward, Right',
        'FWL': 'Forward, Left',
        'DMR': 'Defensive Midfielder, Right',
        'DML': 'Defensive Midfielder, Left',
    }

    return positions.get(abbr, 'Unknown')

In [30]:
df_appearances['position'] = df_appearances['position'].apply(getPositionName)

In [31]:
cursor.execute("DROP TABLE IF EXISTS appearances")
cursor.execute("""
CREATE TABLE appearances (
               ID int primary key,
               gameID int,
               playerID int,
               goals int,
               ownGoals int,
               shots int,
               assists int,
               keyPasses int,
               position varchar(30),
               positionOrder int,
               yellowCard int,
               redCard int,
               time int,
               leagueID int
)
""")

In [32]:
sql = ( """
        INSERT INTO appearances
        VALUES (%(ID)s, %(gameID)s, %(playerID)s, %(goals)s, %(ownGoals)s, %(shots)s, %(assists)s, %(keyPasses)s, %(position)s, %(positionOrder)s, %(yellowCard)s, %(redCard)s, %(time)s, %(leagueID)s)
        """
      ) 
data = df_appearances.to_dict(orient='records')
cursor.executemany(sql, data)
conn.commit()

In [33]:
cursor.execute("drop table if exists appearances_compact")
cursor.execute("""
CREATE TABLE `appearances_compact` (
  `ID` int NOT NULL,
  `gameID` int DEFAULT NULL,
  `playerID` int DEFAULT NULL,
  `goals` int DEFAULT NULL,
  `ownGoals` int DEFAULT NULL,
  `shots` int DEFAULT NULL,
  `assists` int DEFAULT NULL,
  `keyPasses` int DEFAULT NULL,
  `position` int DEFAULT NULL,
  `positionOrder` int DEFAULT NULL,
  `yellowCard` int DEFAULT NULL,
  `redCard` int DEFAULT NULL,
  `time` int DEFAULT NULL,
  `leagueID` int DEFAULT NULL,
  PRIMARY KEY (`ID`)
);
""")

In [34]:
cursor.execute("""
insert into appearances_compact
select ID, gameID, playerID, goals, ownGoals, shots,assists, keyPasses,
case
when position = 'Goalkeeper' then	1
when position = 'Right Defender' then	2
when position = 'Central Defender' then	3
when position = 'Left Defender' then	4
when position = 'Midfielder, Right' then	5
when position = 'Midfielder, Center' then	6
when position = 'Midfielder, Left' then	7
when position = 'Forward' then	8
when position = 'Substitute' then	9
when position = 'Forward, Right' then	10
when position = 'Forward, Left' then	11
when position = 'Defensive Midfielder, Center' then	12
when position = 'Attacking Midfielder, Right' then	13
when position = 'Attacking Midfielder, Center' then	14
when position = 'Attacking Midfielder, Left' then	15
else null
end as position, positionOrder, yellowCard, redCard, time, leagueID from appearances;
""")
conn.commit()

## 5. Left outer join

In [47]:
sql = """SELECT A.ID, A.gameID, A.playerID, a.position, ac.position, ac.positionOrder
        FROM appearances as a LEFT OUTER JOIN appearances_compact as ac
        ON a.ID = ac.ID 
        ORDER BY a.ID ASC
        LIMIT 20;
        """

_, df = dataframe_query(conn, sql)
display(df)

,ID,gameID,playerID,position,position,positionOrder
0,1,81,560,Goalkeeper,1,1
1,2,81,557,Right Defender,2,2
2,3,81,548,Central Defender,3,3
3,4,81,628,Central Defender,3,3
4,5,81,1006,Left Defender,4,4
5,6,81,551,"Defensive Midfielder, Center",12,7
6,7,81,654,"Defensive Midfielder, Center",12,7
7,8,81,554,"Attacking Midfielder, Right",13,11
8,9,81,555,"Attacking Midfielder, Center",14,12
9,10,81,631,"Attacking Midfielder, Left",15,13


### 6. RIGHT OUTER JOIN

In [49]:
sql = """
    SELECT A.ID, A.gameID, A.playerID, a.position, ac.position, ac.positionOrder
        FROM appearances as a RIGHT OUTER JOIN appearances_compact as ac
        ON a.ID = ac.ID 
        ORDER BY a.ID ASC
        LIMIT 20;
        """

_, df = dataframe_query(conn, sql)
display(df)

,ID,gameID,playerID,position,position,positionOrder
0,1,81,560,Goalkeeper,1,1
1,2,81,557,Right Defender,2,2
2,3,81,548,Central Defender,3,3
3,4,81,628,Central Defender,3,3
4,5,81,1006,Left Defender,4,4
5,6,81,551,"Defensive Midfielder, Center",12,7
6,7,81,654,"Defensive Midfielder, Center",12,7
7,8,81,554,"Attacking Midfielder, Right",13,11
8,9,81,555,"Attacking Midfielder, Center",14,12
9,10,81,631,"Attacking Midfielder, Left",15,13


In [ ]:
cursor.close()
conn.close()